In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import librosa
from tqdm import tqdm_notebook as tqdm
import json
import pandas as pd
import pickle
import os
import h5py
import collections as cl
from ast import literal_eval
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score
from sklearn.preprocessing import StandardScaler
import random
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from deem import deem

In [5]:
param_grid = {'LR_param': {'C':[10**k for k in range(-10, 6, 2)]}, 'scoring': 'roc_auc', 'cv': 3, 'random_state': 42}

with open("genre_map.json", "r") as f: # only consider 10 classes of Openmic dataset
    genre_map = json.load(f)
aligned_genre = list(genre_map)[:-1]

with open("instrument_map.json", "r") as f: # only consider 10 classes of Openmic dataset
    instrument_map = json.load(f)

## VGGish

In [6]:
embedding = 'vggish'
debias_method = ''

deb = deem(embedding = embedding, debias_method = debias_method, feature_dir='./embeddings.h5', 
           instrument_map=instrument_map, genre_map=genre_map, param_grid=param_grid)

A_meta = pd.read_csv("A_meta.csv")
A_feature = deb.load_feature(A_meta)
B_meta = pd.read_csv("B_meta.csv")
B_feature = deb.load_feature(B_meta)

deb.instrument_classfication(train_set='A', test_set='A', A_feature=A_feature, B_feature=B_feature)
deb.instrument_classfication(train_set='A', test_set='B', A_feature=A_feature, B_feature=B_feature)
deb.instrument_classfication(train_set='B', test_set='B', A_feature=A_feature, B_feature=B_feature)
deb.instrument_classfication(train_set='B', test_set='A', A_feature=A_feature, B_feature=B_feature)

100%|██████████| 6705/6705 [00:01<00:00, 4428.21it/s]


(6705, 128) (6705,)


100%|██████████| 6705/6705 [00:01<00:00, 4392.16it/s]


(6705, 128) (6705,)
Train on A, test on A


100%|██████████| 11/11 [01:21<00:00,  7.45s/it]


...

100%|██████████| 11/11 [01:14<00:00,  6.78s/it]


Train on B, test on B


100%|██████████| 11/11 [01:06<00:00,  6.04s/it]


Train on B, test on A


100%|██████████| 11/11 [01:04<00:00,  5.89s/it]


In [7]:
debias_method = '-lda'

deb.debias_method = debias_method

deb.instrument_classfication(train_set='A', test_set='A', A_feature=A_feature, B_feature=B_feature)
deb.instrument_classfication(train_set='A', test_set='B', A_feature=A_feature, B_feature=B_feature)
deb.instrument_classfication(train_set='B', test_set='B', A_feature=A_feature, B_feature=B_feature)
deb.instrument_classfication(train_set='B', test_set='A', A_feature=A_feature, B_feature=B_feature)

deb.result_all.to_csv('results/result_' + embedding + debias_method + '.csv', index=False)
result_all = deb.result_all

Train on A, test on A


100%|██████████| 11/11 [01:11<00:00,  6.51s/it]


Train on A, test on B


100%|██████████| 11/11 [01:11<00:00,  6.52s/it]


Train on B, test on B


100%|██████████| 11/11 [01:08<00:00,  6.26s/it]


Train on B, test on A


100%|██████████| 11/11 [01:01<00:00,  5.62s/it]


In [3]:
# embedding = 'vggish'; debias_method = ''
# df1 = pd.read_csv('results/result_' + embedding + debias_method + '.csv')
# debias_method = '-lda'
# df2 = pd.read_csv('results/result_' + embedding + debias_method + '.csv')

# df = df1.append(df2).reset_index(drop=True)
# sns.set_theme(context='notebook', style='whitegrid', palette='deep')

# sns.catplot(data=df, row='train_set', col='test_set', y='instrument', x='roc_auc', hue='embedding', kind='bar', 
#             height=4, aspect=1.2, ci=None)
# plt.xlim([0.6,1])
# plt.savefig('results/' + embedding  + debias_method + '.pdf', bbox_inches='tight')

In [8]:
result_all

,instrument,train_set,test_set,precision,recall,f1-score,support,accuracy,roc_auc,ap,embedding
0,cello,A,A,0.24324324324324326,0.9,0.3829787234042553,30.0,0.8852242744063324,0.9639652014652015,0.6500305880887297,vggish
1,clarinet,A,A,0.2916666666666667,0.8,0.42748091603053434,35.0,0.9010554089709762,0.9426595534479352,0.5259687952775505,vggish
2,flute,A,A,0.25,0.6363636363636364,0.358974358974359,44.0,0.8680738786279684,0.8262668703845175,0.5271853636789754,vggish
3,guitar-ac,A,A,0.6095890410958904,0.9081632653061225,0.7295081967213115,98.0,0.9129287598944591,0.9769480519480519,0.8984266274071752,vggish
4,guitar-el,A,A,0.5124378109452736,0.7923076923076923,0.6223564954682779,130.0,0.8350923482849604,0.912359137677609,0.763658512289708,vggish
...,...,...,...,...,...,...,...,...,...,...,...
83,piano,B,A,0.5882352941176471,0.5128205128205128,0.547945205479452,39.0,0.9564643799472295,0.9094540137655576,0.5843435829823466,vggish-lda
84,saxophone,B,A,0.33962264150943394,0.5142857142857142,0.40909090909090906,35.0,0.9313984168865436,0.7839557399723375,0.3564544531308062,vggish-lda
85,trumpet,B,A,0.3958333333333333,0.4634146341463415,0.42696629213483145,41.0,0.9327176781002638,0.8043337755553288,0.47639371507496725,vggish-lda
86,violin,B,A,0.26595744680851063,0.5434782608695652,0.3571428571428571,46.0,0.8812664907651715,0.7665486077186127,0.41807911778514184,vggish-lda


## OpenL3

In [9]:
embedding = 'openl3'
debias_method = ''

deb = deem(embedding = embedding, debias_method = debias_method, feature_dir='./embeddings.h5', 
           instrument_map=instrument_map, genre_map=genre_map, param_grid=param_grid)

A_meta = pd.read_csv("A_meta.csv")
A_feature = deb.load_feature(A_meta)
B_meta = pd.read_csv("B_meta.csv")
B_feature = deb.load_feature(B_meta)

deb.instrument_classfication(train_set='A', test_set='A', A_feature=A_feature, B_feature=B_feature)
deb.instrument_classfication(train_set='A', test_set='B', A_feature=A_feature, B_feature=B_feature)
deb.instrument_classfication(train_set='B', test_set='B', A_feature=A_feature, B_feature=B_feature)
deb.instrument_classfication(train_set='B', test_set='A', A_feature=A_feature, B_feature=B_feature)

100%|██████████| 6705/6705 [00:28<00:00, 231.64it/s]


(6705, 512) (6705,)


100%|██████████| 6705/6705 [00:29<00:00, 230.79it/s]


(6705, 512) (6705,)
Train on A, test on A


100%|██████████| 11/11 [00:44<00:00,  4.02s/it]


...

100%|██████████| 11/11 [00:43<00:00,  3.97s/it]


Train on B, test on B


100%|██████████| 11/11 [00:42<00:00,  3.87s/it]


Train on B, test on A


100%|██████████| 11/11 [00:41<00:00,  3.79s/it]


In [10]:
debias_method = '-lda'

deb.debias_method = debias_method

deb.instrument_classfication(train_set='A', test_set='A', A_feature=A_feature, B_feature=B_feature)
deb.instrument_classfication(train_set='A', test_set='B', A_feature=A_feature, B_feature=B_feature)
deb.instrument_classfication(train_set='B', test_set='B', A_feature=A_feature, B_feature=B_feature)
deb.instrument_classfication(train_set='B', test_set='A', A_feature=A_feature, B_feature=B_feature)

deb.result_all.to_csv('results/result_' + embedding + debias_method + '.csv', index=False)
result_all = result_all.append(deb.result_all)

Train on A, test on A


100%|██████████| 11/11 [00:44<00:00,  4.03s/it]


Train on A, test on B


100%|██████████| 11/11 [00:43<00:00,  3.92s/it]


Train on B, test on B


100%|██████████| 11/11 [00:42<00:00,  3.83s/it]


Train on B, test on A


100%|██████████| 11/11 [00:41<00:00,  3.78s/it]


In [11]:
# embedding = 'openl3'; debias_method = ''
# df1 = pd.read_csv('results/result_' + embedding + debias_method + '.csv')
# debias_method = '-lda'
# df2 = pd.read_csv('results/result_' + embedding + debias_method + '.csv')

# df = df1.append(df2).reset_index(drop=True)
# sns.set_theme(context='notebook', style='whitegrid', palette='deep')

# sns.catplot(data=df, row='train_set', col='test_set', y='instrument', x='roc_auc', hue='embedding', kind='bar', 
#             height=4, aspect=1.2, ci=None)
# plt.xlim([0.6,1])
# plt.savefig('results/' + embedding  + debias_method + '.pdf', bbox_inches='tight')

## YAMNet

In [12]:
embedding = 'yamnet'
debias_method = ''

deb = deem(embedding = embedding, debias_method = debias_method, feature_dir='./embeddings.h5', 
           instrument_map=instrument_map, genre_map=genre_map, param_grid=param_grid)

A_meta = pd.read_csv("A_meta.csv")
A_feature = deb.load_feature(A_meta)
B_meta = pd.read_csv("B_meta.csv")
B_feature = deb.load_feature(B_meta)

deb.instrument_classfication(train_set='A', test_set='A', A_feature=A_feature, B_feature=B_feature)
deb.instrument_classfication(train_set='A', test_set='B', A_feature=A_feature, B_feature=B_feature)
deb.instrument_classfication(train_set='B', test_set='B', A_feature=A_feature, B_feature=B_feature)
deb.instrument_classfication(train_set='B', test_set='A', A_feature=A_feature, B_feature=B_feature)

100%|██████████| 6705/6705 [00:04<00:00, 1640.14it/s]


(6705, 1024) (6705,)


100%|██████████| 6705/6705 [00:03<00:00, 1713.00it/s]


(6705, 1024) (6705,)
Train on A, test on A


100%|██████████| 11/11 [00:46<00:00,  4.21s/it]


...

100%|██████████| 11/11 [00:45<00:00,  4.14s/it]


Train on B, test on B


100%|██████████| 11/11 [00:38<00:00,  3.52s/it]


Train on B, test on A


100%|██████████| 11/11 [00:38<00:00,  3.50s/it]


In [13]:
debias_method = '-lda'

deb.debias_method = debias_method

deb.instrument_classfication(train_set='A', test_set='A', A_feature=A_feature, B_feature=B_feature)
deb.instrument_classfication(train_set='A', test_set='B', A_feature=A_feature, B_feature=B_feature)
deb.instrument_classfication(train_set='B', test_set='B', A_feature=A_feature, B_feature=B_feature)
deb.instrument_classfication(train_set='B', test_set='A', A_feature=A_feature, B_feature=B_feature)

deb.result_all.to_csv('results/result_' + embedding + debias_method + '.csv', index=False)
result_all = result_all.append(deb.result_all)

Train on A, test on A


100%|██████████| 11/11 [01:31<00:00,  8.32s/it]


Train on A, test on B


100%|██████████| 11/11 [01:20<00:00,  7.31s/it]


Train on B, test on B


100%|██████████| 11/11 [01:16<00:00,  6.98s/it]


Train on B, test on A


100%|██████████| 11/11 [01:15<00:00,  6.88s/it]


In [1]:
# embedding = 'yamnet'; debias_method = ''
# df1 = pd.read_csv('results/result_' + embedding + debias_method + '.csv')
# debias_method = '-lda'
# df2 = pd.read_csv('results/result_' + embedding + debias_method + '.csv')

# df = df1.append(df2)
# sns.set_theme(context='notebook', style='whitegrid', palette='deep')

# sns.catplot(data=df, row='train_set', col='test_set', y='instrument', x='roc_auc', hue='embedding', kind='bar', 
#             height=4, aspect=1.2, ci=None)
# plt.xlim([0.6,1])
# plt.savefig('results/' + embedding  + debias_method + '.pdf', bbox_inches='tight')

In [14]:
# expected length: 44 * 6 = 264
result_all

,instrument,train_set,test_set,precision,recall,f1-score,support,accuracy,roc_auc,ap,embedding
0,cello,A,A,0.24324324324324326,0.9,0.3829787234042553,30.0,0.8852242744063324,0.9639652014652015,0.6500305880887297,vggish
1,clarinet,A,A,0.2916666666666667,0.8,0.42748091603053434,35.0,0.9010554089709762,0.9426595534479352,0.5259687952775505,vggish
2,flute,A,A,0.25,0.6363636363636364,0.358974358974359,44.0,0.8680738786279684,0.8262668703845175,0.5271853636789754,vggish
3,guitar-ac,A,A,0.6095890410958904,0.9081632653061225,0.7295081967213115,98.0,0.9129287598944591,0.9769480519480519,0.8984266274071752,vggish
4,guitar-el,A,A,0.5124378109452736,0.7923076923076923,0.6223564954682779,130.0,0.8350923482849604,0.912359137677609,0.763658512289708,vggish
...,...,...,...,...,...,...,...,...,...,...,...
83,piano,B,A,0.4098360655737705,0.6410256410256411,0.5,39.0,0.9340369393139841,0.9098462964944188,0.6021810146273744,yamnet-lda
84,saxophone,B,A,0.33962264150943394,0.5142857142857142,0.40909090909090906,35.0,0.9313984168865436,0.8227227820588816,0.3344045098319385,yamnet-lda
85,trumpet,B,A,0.4411764705882353,0.7317073170731707,0.5504587155963303,41.0,0.9353562005277045,0.9244140558560396,0.6880225781600884,yamnet-lda
86,violin,B,A,0.30952380952380953,0.2826086956521739,0.29545454545454547,46.0,0.9182058047493403,0.757785784074255,0.2799666420101389,yamnet-lda


In [15]:
result_all.to_csv('results/result_all_lda.csv', index=False)